In [ ]:
import numpy as np
from STL10 import read_all_images, read_labels
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.feature import hog, local_binary_pattern

In [ ]:
 # General data
X_train_imgs = read_all_images("stl10_binary/train_X.bin")
y_train = read_labels("stl10_binary/train_y.bin")

X_test_imgs = read_all_images("stl10_binary/test_X.bin")
y_test = read_labels("stl10_binary/test_y.bin")


 # Raw pixel values
X_train_raw = X_train_imgs[:,:,:,0].reshape( (X_train_imgs.shape[0], X_train_imgs.shape[1] * X_train_imgs.shape[2]) )
X_test_raw  = X_test_imgs[:,:,:,0].reshape(  (X_test_imgs.shape[0],   X_test_imgs.shape[1] * X_test_imgs.shape[2]) ) 

print("raw:", X_train_raw.shape, X_test_raw.shape)


 # Hog & LBP
radius = 3
n_points = 8 * radius    

X_train_hog = []
X_test_hog = []
X_train_lbp = []
X_test_lbp = []

for img in X_train_imgs[:,:,:,0]:
    X_train_hog.append(hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=False))
    X_train_lbp.append(local_binary_pattern(img, n_points, radius, method='default'))

    
for img in X_test_imgs[:,:,:,0]:
    X_test_hog.append(hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=False, multichannel=False))
    X_test_lbp.append(local_binary_pattern(img, n_points, radius, method='default'))    

X_train_hog = np.array(X_train_hog) / np.max(X_train_hog)
X_test_hog = np.array(X_test_hog) /np.max(X_test_hog)

X_train_lbp = np.array(X_train_lbp) / np.max(X_train_lbp)
X_test_lbp  = np.array(X_test_lbp) / np.max(X_test_lbp)

X_train_lbp = X_train_lbp.reshape(X_train_lbp.shape[0], X_train_lbp.shape[1]*X_train_lbp.shape[2] ) 
X_test_lbp  = X_test_lbp.reshape(X_test_lbp.shape[0], X_test_lbp.shape[1]*X_test_lbp.shape[2] )

print("hog:", X_train_hog.shape, X_test_hog.shape)
print("lbp:", X_train_lbp.shape, X_test_lbp.shape)

 # Labels

num_categories = 10
y_train = tf.keras.utils.to_categorical(y_train-1, num_categories)
y_test = tf.keras.utils.to_categorical(y_test-1, num_categories)



In [164]:
def trainTestModel(X_trn, y_trn, X_tst, y_tst, steps, nodes):
        
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units = nodes, activation='sigmoid', input_shape=(X_trn.shape[1],)),
        tf.keras.layers.Dense(units = nodes, activation='tanh'),
        tf.keras.layers.Dense(units = 10, activation="softmax") 
    ])
    #model.summary()
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_trn, y_trn, epochs=steps, verbose=0)
    model.evaluate(X_tst,  y_tst, verbose=1, steps=steps)



In [159]:
nodes = [10,25,50]
steps = [10,25,50]

In [ ]:
 # Raw
for s in steps:
    for i in nodes:
        trainTestModel(X_train_raw, y_train, X_test_raw, y_test, s, i)
        print("Nodes= %d, Epochs = %d,  softmax, tanh and sigmoid " % (i,s))


In [167]:
 # Hog
for s in steps:
    for i in nodes:
        trainTestModel(X_train_hog, y_train, X_test_hog, y_test, s, i)
        print("Nodes= %d, Epochs = %d,  softmax, tanh and sigmoid " % (i,s))


10/10 [==============================] - 0s 2ms/step - loss: 1.6437 - accuracy: 0.3644
Nodes= 10, Epochs = 10,  softmax, tanh and sigmoid 
10/10 [==============================] - 0s 1ms/step - loss: 1.5450 - accuracy: 0.4140
Nodes= 25, Epochs = 10,  softmax, tanh and sigmoid 
10/10 [==============================] - 0s 2ms/step - loss: 1.6023 - accuracy: 0.3851
Nodes= 50, Epochs = 10,  softmax, tanh and sigmoid 
25/25 [==============================] - 0s 1ms/step - loss: 1.5358 - accuracy: 0.4038
Nodes= 10, Epochs = 25,  softmax, tanh and sigmoid 
25/25 [==============================] - 0s 977us/step - loss: 1.4873 - accuracy: 0.4363
Nodes= 25, Epochs = 25,  softmax, tanh and sigmoid 
25/25 [==============================] - 0s 1ms/step - loss: 1.4748 - accuracy: 0.4544
Nodes= 50, Epochs = 25,  softmax, tanh and sigmoid 
50/50 [==============================] - 0s 799us/step - loss: 1.5010 - accuracy: 0.4423
Nodes= 10, Epochs = 50,  softmax, tanh and sigmoid 
50/50 [================

In [166]:
 # LBP
for s in steps:
    for i in nodes:
        trainTestModel(X_train_lbp, y_train, X_test_lbp, y_test, s, i)
        print("Nodes= %d, Epochs = %d,  softmax, tanh and sigmoid " % (i,s))


10/10 [==============================] - 0s 7ms/step - loss: 2.0567 - accuracy: 0.2091
Nodes= 10, Epochs = 10,  softmax, tanh and sigmoid 
10/10 [==============================] - 0s 12ms/step - loss: 2.0668 - accuracy: 0.1991
Nodes= 25, Epochs = 10,  softmax, tanh and sigmoid 
10/10 [==============================] - 0s 11ms/step - loss: 2.1466 - accuracy: 0.2081
Nodes= 50, Epochs = 10,  softmax, tanh and sigmoid 
25/25 [==============================] - 0s 3ms/step - loss: 2.3596 - accuracy: 0.1959
Nodes= 10, Epochs = 25,  softmax, tanh and sigmoid 
25/25 [==============================] - 0s 4ms/step - loss: 3.3189 - accuracy: 0.1937
Nodes= 25, Epochs = 25,  softmax, tanh and sigmoid 
25/25 [==============================] - 0s 5ms/step - loss: 3.7138 - accuracy: 0.2170
Nodes= 50, Epochs = 25,  softmax, tanh and sigmoid 
50/50 [==============================] - 0s 2ms/step - loss: 3.8218 - accuracy: 0.2055
Nodes= 10, Epochs = 50,  softmax, tanh and sigmoid 
50/50 [==================